In [2]:
cd kubeflow-example

/home/jupyter/kubeflow-example


In [22]:
!pwd


/home/jupyter/kubeflow-example


In [24]:
ls -la

total 660
drwxr-xr-x  4 jupyter jupyter   4096 May 12 21:47 ./
drwxr-xr-x 13 jupyter jupyter   4096 May 12 21:41 ../
-rw-r--r--  1 jupyter jupyter     90 May 12 21:47 .dockerignore
drwxr-xr-x  2 jupyter jupyter   4096 May 12 21:47 .ipynb_checkpoints/
-rw-r--r--  1 jupyter jupyter    790 May 12 21:11 Dockerfile
drwxr-xr-x  2 jupyter jupyter   4096 May 12 20:34 __pycache__/
-rw-r--r--  1 jupyter jupyter    740 May 12 21:30 anamoly-detection-kubeflow-sdk-demo.zip
-rw-r--r--  1 jupyter jupyter    269 May 12 21:11 config.py
-rw-r--r--  1 jupyter jupyter    644 May 12 07:12 get_data.py
-rw-r--r--  1 jupyter jupyter 619892 May 12 20:48 isolation_forest_model
-rw-r--r--  1 jupyter jupyter    529 May 12 21:38 preprocess_data.py
-rw-r--r--  1 jupyter jupyter    186 May 12 20:32 requirements.txt
-rw-r--r--  1 jupyter jupyter   7169 May 12 20:49 train.py


In [5]:
%%writefile config.py
gs_bucket_name="test-project-one-307022_bucket"
Bucket_uri="gs://test-project-one-307022_bucket"
version=1
store_artifacts=Bucket_uri + "/" + str(version)
data_path=Bucket_uri + "/" + "data/data_raw.csv"
processed_data=Bucket_uri + "/" + "processed/data_processed.csv"

Overwriting config.py


In [25]:
%%writefile Dockerfile
FROM gcr.io/deeplearning-platform-release/base-cpu
# Install apt dependencies
RUN apt-get update && apt-get install -y \
    git \
    gpg-agent \
    python3-cairocffi \
    protobuf-compiler \
    python3-pil \
    python3-lxml \
    python3-tk \
    wget
# Install gcloud and gsutil commands
# https://cloud.google.com/sdk/docs/quickstart-debian-ubuntu
RUN export CLOUD_SDK_REPO="cloud-sdk-$(lsb_release -c -s)" && \
    echo "deb http://packages.cloud.google.com/apt $CLOUD_SDK_REPO main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list && \
    curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - && \
    apt-get update -y && apt-get install google-cloud-sdk -y
WORKDIR /pipeline
COPY ./ ./
RUN pip install -r requirements.txt
ENV TF_CPP_MIN_LOG_LEVEL 3

Overwriting Dockerfile


In [26]:
!pwd

/home/jupyter/kubeflow-example


In [27]:
!gcloud builds submit --timeout=900s --tag gcr.io/test-project-one-307022/trainer-image:v1


Creating temporary tarball archive of 17 file(s) totalling 625.4 KiB before compression.
Uploading tarball of [.] to [gs://test-project-one-307022_cloudbuild/source/1620856114.907588-6e0422be9675492bb9be92bbe5e90ace.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/test-project-one-307022/locations/global/builds/82437372-66f7-408f-a2e5-d4f5cc833e41].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/82437372-66f7-408f-a2e5-d4f5cc833e41?project=1000853722526].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "82437372-66f7-408f-a2e5-d4f5cc833e41"

FETCHSOURCE
Fetching storage object: gs://test-project-one-307022_cloudbuild/source/1620856114.907588-6e0422be9675492bb9be92bbe5e90ace.tgz#1620856115497382
Copying gs://test-project-one-307022_cloudbuild/source/1620856114.907588-6e0422be9675492bb9be92bbe5e90ace.tgz#1620856115497382...
/ [1 files][207.0 KiB/207.0 KiB]                                               

In [9]:
!pip install kfp

In [28]:
import kfp
import kfp.components as comp
from kubernetes.client.models import V1EnvVar

In [29]:
@kfp.dsl.component
def get_data():
    # Defining component configuration
    getdata_component = kfp.dsl.ContainerOp(
        name='Data-Preparation',
        image='gcr.io/test-project-one-307022/trainer-image:v1',
        command=['python', 'get_data.py'],
        )
    return getdata_component

In [30]:
@kfp.dsl.component
def data_preprocessing():    
    # Defining component configuration
    data_preprocessing = kfp.dsl.ContainerOp(
        name='data-preprocessing',
        image='gcr.io/test-project-one-307022/trainer-image:v1',
        command=['python', 'preprocess_data.py'],
        )
    return data_preprocessing

In [31]:
@kfp.dsl.component
def training():
    # Defining component configuration
    training_component = kfp.dsl.ContainerOp(
        name='training',
        image='gcr.io/test-project-one-307022/trainer-image:v1',
        command=['python', 'train.py'],
        file_outputs={'mlpipeline-ui-metadata':'/mlpipeline-ui-metadata.json', "mlpipeline-metrics":'/mlpipeline-metrics.json'}
        )
    
    return training_component

In [32]:
# Let see output of component configuration
debug = True
if debug :
    training_component_vis = training()
    print(training_component_vis)

{'ContainerOp': {'is_exit_handler': False, 'human_name': 'training', 'display_name': None, 'name': 'training 800007f97736be41', 'node_selector': {}, 'volumes': [], 'tolerations': [], 'affinity': {}, 'pod_annotations': {}, 'pod_labels': {}, 'retry_policy': None, 'num_retries': 0, 'timeout': 0, 'init_containers': [], 'sidecars': [], 'loop_args': None, '_inputs': [], 'dependent_names': [], 'attrs_with_pipelineparams': ['node_selector', 'volumes', 'pod_annotations', 'pod_labels', 'num_retries', 'init_containers', 'sidecars', 'tolerations', '_container', 'artifact_arguments', '_parameter_arguments'], '_container': {'args': None,
 'command': ['python', 'train.py'],
 'env': None,
 'env_from': None,
 'image': 'gcr.io/test-project-one-307022/trainer-image:v1',
 'image_pull_policy': None,
 'lifecycle': None,
 'liveness_probe': None,
 'ports': None,
 'readiness_probe': None,
 'resources': None,
 'security_context': None,
 'stdin': None,
 'stdin_once': None,
 'termination_message_path': None,
 'te

Final Kubeflow pipeline Definition

In [33]:
@kfp.dsl.pipeline(
  name="Credit card Fraud-Detection",
  description="Detecting the anamolies in the Credit card transaction data"
)
def anamoly_detection():
    download_data = get_data()
    download_data.execution_options.caching_strategy.max_cache_staleness = "P0D"
    data_processing = data_preprocessing().after(download_data)
    data_processing.execution_options.caching_strategy.max_cache_staleness = "P0D"
    train = training().after(data_processing)
    train.execution_options.caching_strategy.max_cache_staleness = "P0D"

In [34]:


# Let see output of pipeline configuration
debug = True
if debug :
    training_pipeline_output = anamoly_detection()
    print(training_pipeline_output)



None



Compile Kubeflow Pipeline

    It will Generate .zip file inside this contain YAMl file which contain the configuration of kubeflow pipeline



In [35]:
kfp.compiler.Compiler().compile(anamoly_detection, 'anamoly-detection-kubeflow-sdk-demo.zip')

Connect to deployed kubeflow pipeline Endpoint (GCP)

In [36]:
# Create kfp client
# Note: Add the KubeFlow Pipeline endpoint below if the client is not running on the same cluster.
client = kfp.Client("2af2fec3eaa286dd-dot-us-west1.pipelines.googleusercontent.com")


Create Experiment


In [37]:
EXPERIMENT_NAME = 'anamoly-detection'
experiment = client.create_experiment(name=EXPERIMENT_NAME)

In [38]:

# Deploy pipeline to kubeflow pipeline Endpoint


In [39]:
run = client.run_pipeline(experiment.id, 'anamoly-run-1', 'anamoly-detection-kubeflow-sdk-demo.zip')